In [ ]:
#import
import pandas as pd
import numpy as np
import os
from datetime import datetime

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import warnings
import itertools

import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, minmax_scale

from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest

In [ ]:
#define
SESSION_PATH = './sessions'
WELLNESS_PATH = './wellness'
PICKLE_PATH = './pickle/'
format = '%Y-%m-%d'
lista_colonne_utili_analisi_esercizi = ['splitName',
#                                         'playerName',
                                        'workload',
                                        'perceivedWorkload',
                                        'metabolicWorkload',
                                        'mechanicalWorkload',
                                        'kinematicWorkload',
                                        'cardioWorkload']

#colonne da shiftare verso l'alto

lista_colonne_workload = [
    'workload-1',
    'workload-2',
    'workload-3',
    'workload-4',
    'workload-5'
                        ]
lista_colonne_rpe = [
    'rpe-1',
    'rpe-2',
    'rpe-3',
    'rpe-4',
    'rpe-5'
]
lista_colonne_Go_score = [
    'Go_score-1',
    'Go_score-2',
    'Go_score-3',
    'Go_score-4',
    'Go_score-5'
]

######################################
lista_category = [[lista_colonne_workload,'workload'],[lista_colonne_rpe,'rpe'],[lista_colonne_Go_score,'Go_score']]
######################################
#colonne per DNN


#pesi per calcolare GO_Score

SLEEP_PAR = [0.1, 0.1, 0.5, 0.8, 1]
STRESS_PAR = [0.1, 0.1, 1, 1, 1]
FATIGUE_PAR = [0.1, 0.1, 0.5, 0.8, 1]
SORENESS_PAR = [0.1, 0.1, 0.5, 0.8, 1]
MOOD_PAR = [0.1, 0.1, 1, 1, 1]

#script per calcolare nomi colonne
nome = 'Go_score'
n = 5
lista = []
for i in range(-n,0):
    lista.append(nome+str(i))
reversed(lista)

In [ ]:
def from_str_to_date(stringa):
    timestr = stringa.split(' ')[0]
    time = datetime.strptime(timestr, format)
    return time

def calcola_go(df):
    df['Go_score'] = SLEEP_PAR[int(df['wellness_sleep']) - 1] + STRESS_PAR[int(df['wellness_stress']) - 1] + FATIGUE_PAR[int(df['wellness_fatigue']) - 1] + SORENESS_PAR[int(df['wellness_soreness']) - 1] + MOOD_PAR[int(df['wellness_mood']) - 1]
    return df

CREO UN DB CON I DATI DI TUTTI I GIOCATORI, CON COLONNE RIFERITE AI GIORNI PRECEDENTI
===========================

In [ ]:
# importo csv associati a ogni player
list_files1 = os.listdir(PICKLE_PATH)
list_file = list_files1.copy()

# tolgo dalla lista dei files quelli non di player
for file in list_files1:
    if 'Player' not in file:
        list_file.remove(file)
    elif 'csv' in file:
        list_file.remove(file)

df_total = pd.DataFrame()
#per ogni file, creo e preparo il db da dare al DNN
for file in list_file:
    print(file)
    player = file.split('.')[0]
    globals()[player] = pd.read_pickle(PICKLE_PATH + file)
    
    #calcolo go score
    df_player = globals()[player].apply(calcola_go, axis=1)

    #drop delle colonne inutili
    df_player = df_player.drop(['exercises','playerName','wellness_fatigue','wellness_mood','complete',
                                'wellness_sleep','wellness_soreness','wellness_stress','Work Ratio','rpeTl','duration'], axis=1)
    
    ######################################################################################################
    for category in lista_category:
        i = 1
        for column in category[0]:
            df_player[column] = df_player[category[1]].shift(i)
            i += 1

    
    df_player = df_player.dropna(how='any')
    
    globals()[player] = df_player.copy()
    df_total = df_total.append(df_player, ignore_index=True)

#rinomino la colonna Player Load
df_total.rename(columns={'Player Load': 'Player_Load'}, inplace=True)

file = list_file[0]
player = file.split('.')[0]
globals()[player] = pd.read_pickle(PICKLE_PATH + file)

#calcolo go score
df_player = globals()[player].apply(calcola_go, axis=1)

#drop delle colonne inutili
df_player = df_player.drop(['exercises','playerName','wellness_fatigue','wellness_mood','complete',
                            'wellness_sleep','wellness_soreness','wellness_stress','Work Ratio','rpeTl','duration'], axis=1)
df_player

#creo colonne per i workload
i = 1
for column in lista_colonne_workload:
    df_player[column] = df_player['workload'].shift(i)
    i += 1

#creo colonne per i rpe
i = 1
for column in lista_colonne_rpe:
    df_player[column] = df_player['rpe'].shift(i)
    i += 1

#creo colonne per i Go_score
i = 1
for column in lista_colonne_Go_score:
    df_player[column] = df_player['Go_score'].shift(i)
    i += 1

df_player.dropna(how='any')

In [ ]:
df_total

In [ ]:
df_total.columns.values

In [ ]:
df_total['Go_score'].value_counts()

RETE NEURALE 
============================

In [ ]:
#define
OUTPUT = 'Go_score'
EPOCHS = 400

In [ ]:
train = df_total.drop(['date','cardioWorkload','intensity','Player_Load'],axis = 1)
train = train[train['workload']!=0]
input_dataset = train.copy()
train

In [ ]:
# Isolate outliers with an IsolationForest algorithm

'''
clf = IsolationForest(max_samples=100, random_state=42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns=['Top'])
y_noano[y_noano['Top'] == 1].index.values
train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop=True, inplace=True)
print('')
print('Number of Outliers:', y_noano[y_noano['Top'] == -1].shape[0])
print('Number of rows without outliers:', train.shape[0])
'''

# Preprocessing dataset with MinMaxScale function

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove(OUTPUT)

# mat_train = np.matrix(train)
# #mat_input = np.matrix(input_dataset)
# mat_new = np.matrix(train.drop(OUTPUT, axis=1))
# mat_y = np.array(train[OUTPUT].reshape((train.shape[0], 1)))

# prepro_y = MinMaxScaler()
# prepro_y.fit(mat_y)

# prepro = MinMaxScaler()
# prepro.fit(mat_train)

# prepro_input = MinMaxScaler()
# prepro_input.fit(mat_new)

mat_train = train.copy()
prepro = MinMaxScaler()
prepro.fit(mat_train)

mat_y = train[OUTPUT].reshape(-1, 1) 
prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

train = pd.DataFrame(prepro.transform(mat_train), columns=col_train)

#input_dataset = pd.DataFrame(prepro_input.transform(mat_input), columns=col_train_bis)

# TensorFlow deep neural network

# List of features

COLUMNS = col_train
FEATURES = col_train_bis

# Columns for tensorflow

feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

# Training set and Prediction set with the features to predict

training_set = train[COLUMNS]
prediction_set = train[OUTPUT]

# Train and Test split


x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES], prediction_set, test_size=0.1, random_state=42)
y_train = pd.DataFrame(y_train, columns=[OUTPUT])
training_set = pd.DataFrame(x_train, columns=FEATURES).merge(y_train, left_index=True, right_index=True)


# Training for submission


training_sub = training_set[col_train]
y_test = pd.DataFrame(y_test, columns=[OUTPUT])
testing_set = pd.DataFrame(x_test, columns=FEATURES).merge(y_test, left_index=True, right_index=True)


# Model

tf.logging.set_verbosity(tf.logging.INFO)
regressor = tf.estimator.DNNRegressor(hidden_units=[20,10], feature_columns=feature_cols, activation_fn=tf.nn.relu) # model_dir = 'regressor'

# Reset the index of training

training_set.reset_index(drop=True, inplace=True)

# Deep Neural Network Regressor with the training set which contain the data split by train test split

regressor.train(input_fn=tf.estimator.inputs.pandas_input_fn(x = training_set, y = training_set[OUTPUT], 
                                                             shuffle=False), steps=EPOCHS)

# Evaluation on the test set created by train_test_split


ev = regressor.evaluate(input_fn=lambda: input_fn(testing_set), steps=1)


# Display the score on the testing set


loss_score1 = ev['loss']
print('')
print('Final Loss on the testing set: {0:f}'.format(loss_score1))
print('')

# Predictions on testing set


y = regressor.predict(input_fn=tf.estimator.inputs.pandas_input_fn(testing_set, shuffle=False))
predictions = list(itertools.islice(y, testing_set.shape[0]))


# Plot predictions x reality on dataset graph


predictions = prepro_y.inverse_transform(np.array([x['predictions'] for x in predictions]).reshape(len(x_test), 1))
reality = pd.DataFrame(prepro.inverse_transform(testing_set), columns=[COLUMNS])[OUTPUT].values
font = {'family' : 'normal',
        'size'   : 10}
matplotlib.rc('font', **font)
fig, ax = plt.subplots(figsize=(50, 40))
axes = plt.gca()
axes.set_xlim([-1, 5])
axes.set_ylim([-1, 5])
plt.style.use('default')
plt.plot(predictions, reality, 'ro', color='red')
plt.xlabel('Predictions', fontsize=20)
plt.ylabel('Reality', fontsize=20)
plt.title('Predictions x Reality on dataset', fontsize=30)
ax.plot([-1, 4.875], [-1, 4.875], 'k--', lw=4)
plt.show()


# Prediction on input dataset

'''y_predict = regressor.predict(input_fn=lambda: input_fn(input_dataset, pred=True))

def to_submit(pred_y):

    y_predict2 = list(itertools.islice(pred_y, input_dataset.shape[0]))
    y_predict2 = pd.DataFrame(prepro_y.inverse_transform(np.array([x['predictions'] for x in y_predict2]).reshape(len(y_predict2), 1)), columns=[OUTPUT])

    return y_predict2

results = to_submit(y_predict)

print('')
print(results) '''

In [ ]:
prepro_y.scale_

In [ ]:
train.head()